In [17]:
# import numpy as np
# import glob
# import jsonlines
# import os
# from pathlib import Path
# import json

import os 
import json
import cv2
import numpy as np
import clip
import torch
from PIL import Image
from tqdm import tqdm
import pathlib
from pathlib import Path
import glob
import argparse
import json
import jsonlines
import json_numpy

import pandas as pd
import pyarrow as pa
import time

# df = pd.DataFrame({'one': [-1, np.nan, 2.5],
#                    'two': ['foo', 'bar', 'baz'],
#                    'three': [True, False, True]},
#                    index=list('abc'))


# table = pa.Table.from_pandas(df)

In [2]:
import pyarrow.parquet as pq

pq.write_table(table, 'example2.parquet')

In [5]:
table2 = pq.read_table('example.parquet')
table2.to_pandas()

,one,two,three
a,-1.0,foo,True
b,NaN,bar,False
c,2.5,baz,True


In [3]:
parquet_file = pq.ParquetFile('example.parquet')
metadata = parquet_file.metadata
metadata

  created_by: parquet-cpp-arrow version 10.0.0
  num_columns: 4
  num_rows: 3
  num_row_groups: 1
  format_version: 2.6
  serialized_size: 2576

In [13]:
video_data_path = '/scratch/bbki/kastanday/whisper/parallel_12'
video_input_dir = Path(video_data_path)
output_path = str(os.path.join(video_input_dir.parent, video_input_dir.stem + '_clip_output.jsonl'))
audio_jsonl = str(os.path.join(video_input_dir.parent, video_input_dir.stem + '_whisper_output.jsonl'))

In [14]:
audio_jsonl

'/scratch/bbki/kastanday/whisper/parallel_12_whisper_output.jsonl'

In [18]:
video_dir_files = set()
stem_to_filename = {}

for video_dir_file in glob.glob(os.path.join(video_data_path, '*'), recursive = True):
    # print(Path(video_dir_file).stem)
    video_dir_files.add(str(Path(video_dir_file).stem))
    stem_to_filename[str(Path(video_dir_file).stem)] = video_dir_file

start_time = time.monotonic()
audio_file_stems = []
stem_to_whisper = {}  # filename.stem --> whisper_json_object
with jsonlines.open(audio_jsonl) as reader:
    for _, obj in enumerate(reader):
        json_objs = json.loads(obj)
        for json_obj in json_objs:
            audio_file_stems.append(json_obj['video_filename_stem'])

            if json_obj['video_filename_stem'] not in stem_to_whisper:
                stem_to_whisper[json_obj['video_filename_stem']] = []
            
            stem_to_whisper[json_obj['video_filename_stem']].append(json_obj)

print(f"runtime: {time.monotonic() - start_time:2f} seconds")
stem_to_whisper

InvalidLineError: line contains invalid json: Extra data: line 1 column 271291 (char 271290) (line 13602)

In [19]:
! cp /scratch/bbki/kastanday/whisper/parallel_12_whisper_output.jsonl /tmp